# Use AutoAI with Watson Studio project `ibm-watsonx-ai`

This notebook contains the steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning service inside Watson Studio's projects. It introduces commands for data retrieval, training experiments and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goals

The learning goals of this notebook are:

-  Work with Watson Machine Learning experiments to train AutoAI model using Watson Studio project.
-  Online, Batch deployment and score the trained model trained.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Optimizer definition](#definition)
3. [Experiment Run](#run)
4. [Deploy and Score](#scoring)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have space already created follow bellow steps.

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
project_id = 'PASTE YOUR PROJECT ID HERE'

To be able to interact with all resources available in Watson Machine Learning, you need to set the **project** which you will be using.

In [5]:
client.set.default_project(project_id)

'SUCCESS'

<a id="definition"></a>
## 2. Optimizer definition

### Training data connection

Define connection information to training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd5.1/data/credit_risk/german_credit_data_biased_training.csv).

In [ ]:
import os, wget

filename = 'german_credit_data_biased_training.csv'
base_url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd5.1/data/credit_risk/'

if not os.path.isfile(filename): wget.download(base_url + filename)

In [7]:
asset_details = client.data_assets.create('german_credit_data_biased_training', filename)
asset_details

Creating data asset...
SUCCESS


{'metadata': {'project_id': '9045160b-4872-4649-85fa-254cb5072dad',
  'sandbox_id': '9045160b-4872-4649-85fa-254cb5072dad',
  'usage': {'last_updated_at': '2024-04-29T12:32:23Z',
   'last_updater_id': '1000330999',
   'last_update_time': 1714393943268,
   'last_accessed_at': '2024-04-29T12:32:23Z',
   'last_access_time': 1714393943268,
   'last_accessor_id': '1000330999',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'1000330999': {'user_iam_id': '1000330999',
     'roles': ['OWNER']}}},
  'name': 'german_credit_data_biased_training',
  'description': '',
  'asset_type': 'data_asset',
  'origin_country': 'us',
  'resource_key': 'german_credit_data_biased_training',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '885bace0-163c-43ac-ad38-7ea28364230c',
  'created': 1714393943268,
  'created_at': '2024-04-29T12:32:23Z',
  'owner_id': '1000330999',
  'size': 0,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['data_

In [8]:
client.data_assets.get_id(asset_details)

'509dd5b4-aa95-40ac-8e47-4dd1511babf7'

In [9]:
from ibm_watsonx_ai.helpers import DataConnection


credit_risk_conn = DataConnection(data_asset_id=client.data_assets.get_id(asset_details))

training_data_reference=[credit_risk_conn]

### Optimizer configuration

Provide the input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric

In [10]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, project_id)

pipeline_optimizer = experiment.optimizer(
    name='Credit Risk Prediction - AutoAI',
    desc='Sample notebook',
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column='Risk',
    scoring=AutoAI.Metrics.ROC_AUC_SCORE,
)

Configuration parameters can be retrieved via `get_params()`.

In [11]:
pipeline_optimizer.get_params()

{'name': 'Credit Risk Prediction - AutoAI',
 'desc': 'Sample notebook',
 'prediction_type': 'binary',
 'prediction_column': 'Risk',
 'prediction_columns': None,
 'timestamp_column_name': None,
 'scoring': 'roc_auc',
 'holdout_size': None,
 'max_num_daub_ensembles': None,
 't_shirt_size': 'm',
 'train_sample_rows_test_size': None,
 'include_only_estimators': None,
 'include_batched_ensemble_estimators': None,
 'backtest_num': None,
 'lookback_window': None,
 'forecast_window': None,
 'backtest_gap_length': None,
 'cognito_transform_names': None,
 'csv_separator': ',',
 'excel_sheet': None,
 'encoding': 'utf-8',
 'positive_label': None,
 'drop_duplicates': True,
 'outliers_columns': None,
 'text_processing': None,
 'word2vec_feature_number': None,
 'daub_give_priority_to_runtime': None,
 'text_columns_names': None,
 'sampling_type': None,
 'sample_size_limit': None,
 'sample_rows_limit': None,
 'sample_percentage_limit': None,
 'number_of_batch_rows': None,
 'n_parallel_data_connections'

<a id="run"></a>
## 3. Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [12]:
run_details = pipeline_optimizer.fit(
            training_data_reference=training_data_reference,
            background_mode=False)

Training job 76b75a8c-1358-424b-95f4-5858ac05e1cc completed: 100%|████████| [02:02<00:00,  1.23s/it]


You can use the `get_run_status()` method to monitor AutoAI jobs in background mode.

In [13]:
pipeline_optimizer.get_run_status()

'completed'

<a id="comparison"></a>
### 3.1 Pipelines comparison

You can list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame by calling the `summary()` method. You can
use the DataFrame to compare all discovered pipelines and select the one
you like for further testing.

In [14]:
summary = pipeline_optimizer.summary()
summary

,Enhancements,Estimator,training_roc_auc_(optimized),holdout_average_precision,holdout_log_loss,training_accuracy,holdout_roc_auc,training_balanced_accuracy,training_f1,holdout_precision,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1
Pipeline Name,,,,,,,,,,,,,,,,,,
Pipeline_7,HPO,XGBClassifier,0.853281,0.479185,0.425331,0.800358,0.832534,0.748267,0.857836,0.816216,0.916755,0.428845,0.909639,0.814358,0.803607,0.751226,0.906382,0.860399
Pipeline_6,,XGBClassifier,0.848451,0.466398,0.356549,0.796120,0.829125,0.749861,0.852969,0.848066,0.912986,0.439419,0.924699,0.818963,0.839679,0.797679,0.890274,0.884726
Pipeline_1,,SnapBoostingMachineClassifier,0.850092,0.468067,0.403035,0.755522,0.825103,0.745869,0.808007,0.893333,0.915222,0.457473,0.807229,0.844414,0.807615,0.807806,0.775171,0.848101
Pipeline_8,"HPO, FE",XGBClassifier,0.853118,0.480923,0.434142,0.801028,0.822542,0.745802,0.859194,0.806366,0.915876,0.428641,0.915663,0.811125,0.797595,0.739268,0.913428,0.857546
Pipeline_9,"HPO, FE, HPO",XGBClassifier,0.853814,0.480976,0.434398,0.800359,0.822379,0.744805,0.858772,0.798942,0.916232,0.428172,0.909639,0.810349,0.787575,0.727274,0.913429,0.850704
Pipeline_10,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(XGBClassifier),0.853814,0.480976,0.434398,0.800359,0.822379,0.744805,0.858772,0.798942,0.916232,0.428172,0.909639,0.810349,0.787575,0.727274,0.913429,0.850704
Pipeline_2,HPO,SnapBoostingMachineClassifier,0.851431,0.477125,0.456636,0.749054,0.822352,0.747928,0.798939,0.883803,0.916453,0.466879,0.756024,0.853730,0.771543,0.779210,0.751345,0.814935
Pipeline_4,"HPO, FE, HPO",SnapBoostingMachineClassifier,0.854760,0.471800,0.427606,0.762663,0.820431,0.753549,0.813832,0.888136,0.917273,0.456183,0.789157,0.849679,0.793587,0.795776,0.781216,0.835726
Pipeline_5,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(SnapBoostingMach...,0.854760,0.471800,0.427606,0.762663,0.820431,0.753549,0.813832,0.888136,0.917273,0.456183,0.789157,0.849679,0.793587,0.795776,0.781216,0.835726


You can visualize the scoring metric calculated on a holdout data set.

In [15]:
import pandas as pd
pd.options.plotting.backend = "plotly"

summary.holdout_roc_auc.plot()

<a id="scoring"></a>
## 4. Deploy and Score

In this section you will learn how to deploy and score trained model using project in a specified deployment space as a webservice and batch using WML instance.

### Webservice deployment creation

In [16]:
from ibm_watsonx_ai.deployment import WebService

service = WebService(
    source_instance_credentials=credentials,
    source_project_id=project_id,
    target_instance_credentials=credentials,
    target_space_id=space_id
)

service.create(
    experiment_run_id=run_details['metadata']['id'],
    model='Pipeline_1', 
    deployment_name="Credit Risk Deployment AutoAI")

Preparing an AutoAI Deployment...
Published model uid: 3a60bec2-5d41-4b2b-9006-34ff20f086b4
Deploying model 3a60bec2-5d41-4b2b-9006-34ff20f086b4 using V4 client.


######################################################################################

Synchronous deployment creation for id: '3a60bec2-5d41-4b2b-9006-34ff20f086b4' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
....
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='0f2fd01b-e941-4532-b9a1-36fded5abbed'
-----------------------------------------------------------------------------------------------




Deployment object could be printed to show basic information:

In [ ]:
print(service)

To show all available information about the deployment use the `.get_params()` method:

In [ ]:
service.get_params()

### Scoring of webservice
You can make scoring request by calling `score()` on deployed pipeline.

In [19]:
pipeline_optimizer.get_data_connections()[0]

{'type': 'data_asset', 'connection': {}, 'location': {'href': '/v2/assets/509dd5b4-aa95-40ac-8e47-4dd1511babf7?project_id=9045160b-4872-4649-85fa-254cb5072dad', 'id': '509dd5b4-aa95-40ac-8e47-4dd1511babf7'}}

In [ ]:
train_df = pipeline_optimizer.get_data_connections()[0].read()

train_X = train_df.drop(['Risk'], axis=1)
train_y = train_df.Risk.values

In [21]:
predictions = service.score(payload=train_X.iloc[:10])
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.6722619492984393, 0.3277380507015607]],
    ['No Risk', [0.7463715340362654, 0.25362846596373456]],
    ['No Risk', [0.8389065655314143, 0.16109343446858565]],
    ['No Risk', [0.5417064635733093, 0.45829353642669074]],
    ['Risk', [0.10046554443553835, 0.8995344555644617]],
    ['Risk', [0.015637843561548737, 0.9843621564384513]],
    ['No Risk', [0.6199513834792403, 0.3800486165207597]],
    ['No Risk', [0.6893342531092224, 0.3106657468907776]],
    ['No Risk', [0.8737911497882562, 0.1262088502117438]],
    ['Risk', [0.02874258944966579, 0.9712574105503342]]]}]}

If you want to work with the web service in an external Python application you can retrieve the service object by:
 - Initialize the service by `service = WebService(wml_credentials)`
 - Get deployment_id by `service.list()` method
 - Get webservice object by `service.get('deployment_id')` method

After that you can call `service.score()` method.

### Deleting deployment
You can delete the existing deployment by calling the `service.delete()` command.
To list the existing web services you can use `service.list()`.

### Batch deployment creation

A batch deployment processes input data from a inline data and return predictions in scoring details or processes from data asset and writes the output to a file.

In [22]:
batch_payload_df = train_df.drop(['Risk'], axis=1)[:5]
batch_payload_df

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,2,savings_insurance,37,stores,own,2,skilled,1,none,yes
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,3,real_estate,28,none,own,2,skilled,1,yes,no
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,2,savings_insurance,32,none,own,1,skilled,1,none,yes
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,3,unknown,57,none,own,2,skilled,1,none,yes


Create batch deployment for `Pipeline_2` created in AutoAI experiment with the `run_id`.

In [23]:
from ibm_watsonx_ai.deployment import Batch

service_batch = Batch(
    source_wml_credentials=credentials,
    source_project_id=project_id,
    target_wml_credentials=credentials,
    target_space_id=space_id
)
service_batch.create(
            experiment_run_id=run_details['metadata']['id'],
            model="Pipeline_2",
            deployment_name="Credit Risk Batch Deployment AutoAI")

Preparing an AutoAI Deployment...
Published model uid: 3ff26359-2da3-4e65-a389-8abae93c8a48
Deploying model 3ff26359-2da3-4e65-a389-8abae93c8a48 using V4 client.


######################################################################################

Synchronous deployment creation for id: '3ff26359-2da3-4e65-a389-8abae93c8a48' started

######################################################################################


ready.


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='58f66c48-1443-4e25-99c2-eef39ae0c13a'
-----------------------------------------------------------------------------------------------




### Score batch deployment with inline payload as pandas DataFrame.

In [24]:
scoring_params = service_batch.run_job(
            payload=batch_payload_df,
            background_mode=False)



##########################################################################

Synchronous scoring for id: 'c09cec17-3db7-44c5-978c-c19c9b33cd36' started

##########################################################################


queued..
completed
Scoring job  'c09cec17-3db7-44c5-978c-c19c9b33cd36' finished successfully.


In [25]:
scoring_params['entity']['scoring'].get('predictions')

[{'fields': ['prediction', 'probability'],
  'values': [['No Risk', [0.6571863089102029, 0.3428136910897971]],
   ['No Risk', [0.7413540815870145, 0.2586459184129854]],
   ['No Risk', [0.8249595432516822, 0.17504045674831784]],
   ['No Risk', [0.597848000493859, 0.402151999506141]],
   ['Risk', [0.18899251320763888, 0.8110074867923611]]]}]

### Deleting deployment
You can delete the existing deployment by calling the `service_batch.delete()` command.
To list the existing: 
- batch services you can use `service_batch.list()`, 
- scoring jobs you can use `service_batch.list_jobs()`.

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps

You successfully completed this notebook!.
 
You learned how to use `watsonx.ai` to run AutoAI experiments using project. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Jan Sołtysik**, Intern in Watson Machine Learning

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning and watsonx.ai.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.